# Generacion Features de Grafo

## Imports & Load Data

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import geopandas as gpd
import networkx as nx

import numpy as np
import pandas as pd

import pickle

In [3]:
# Cargar endpoint
infile = open('./data/1_GDB_Layers.pkl','rb')
GDB = pickle.load(infile)
infile.close()

In [4]:
filename = './data/22_Full_Graph.pkl'
infile = open(filename, 'rb')
MTG = pickle.load(infile)
infile.close()

In [5]:
filename = './data/22_CTMT_Graph_Dict.pkl'
infile = open(filename, 'rb')
CTMT_GR = pickle.load(infile)
infile.close()

In [6]:
filename = './data/22_UNTRD_Conections.pkl'
infile = open(filename, 'rb')
UNTRD_CONS = pickle.load(infile)
infile.close()

## Obtener Rutas a Sub Estacion

In [7]:
def get_route(trd):
    route = []
    
    if trd.UNTRD_PCON == '':
        return route
    
    try:
        route = nx.shortest_path(CTMT_GR[trd.CTMT], trd.UNTRD_PCON, trd.CTMT_PCON, weight='length')
    except Exception as e:
        print('-----------')
        print(e)
        print('Probando con grafo Gral')
        try:
            route = nx.shortest_path(MTG, trd.UNTRD_PCON, trd.CTMT_PCON, weight='length')
        except Exception as e:
            print(e)
            print(f'CTMT:{trd.CTMT}, Source:{trd.UNTRD_PCON}, Target:{trd.CTMT_PCON}')
    
    return route

In [8]:
UNTRD_CONS['ROUTE_SUB'] = UNTRD_CONS.apply(get_route, axis=1)

-----------
No path to nan.
Probando con grafo Gral
No path to nan.
CTMT:27117492, Source:27128440, Target:nan
-----------
No path to 27863987.
Probando con grafo Gral
No path to 27863987.
CTMT:27117487, Source:27148623, Target:27863987
-----------
No path to 27707915.
Probando con grafo Gral
No path to 27707915.
CTMT:27117496, Source:27149709, Target:27707915


In [9]:
## Ruta hasta sub estacion referenciado a los ID de los segmentos de SSDMT
UNTRD_CONS['SID_ROUTE_SUB'] = UNTRD_CONS.apply(
    lambda x: [at['COD_ID'] for p1,p2,at in CTMT_GR[x.CTMT].edges(x.ROUTE_SUB, data=True)], axis=1)

In [10]:
UNTRD_CONS.shape

(3848, 8)

In [11]:
# Pre filtrar variables de interes
SSDMT_ft = ['COD_ID','TIP_CND','COMP']
SEGCON_ft = ['COD_ID', 'R1', 'X1', 'FTRCNV', 'CNOM', 'CMAX'] 

In [12]:
# Cargar valores electricos relacionados a los segmentos de transmicion
SSDMT_ELE = pd.merge(GDB['SSDMT'][SSDMT_ft], GDB['SEGCON'][SEGCON_ft],
                     how='left', left_on='TIP_CND', right_on='COD_ID', suffixes=('','CON'))

## Adicionar Informacion Electrica

In [13]:
def get_elec_features(TRD):
    
    SEG_REF_ROUTE = TRD.SID_ROUTE_SUB
    
    SRR_ELE = SSDMT_ELE[SSDMT_ELE.COD_ID.isin(SEG_REF_ROUTE)]
    
    DISTANCIA = SRR_ELE.COMP.sum()
    RESISTENCIA = (SRR_ELE.R1  * SRR_ELE.COMP).sum()
    REACTANCIA = (SRR_ELE.X1 * SRR_ELE.COMP).sum()
    
    FCONV = SRR_ELE.FTRCNV.mean()
    
    I_NOM_MIN = SRR_ELE.CNOM.min()
    I_NOM_MED = SRR_ELE.CNOM.median()
    
    I_MAX_MIN = SRR_ELE.CMAX.min()
    I_MAX_MED = SRR_ELE.CMAX.median()
    
    elect_ft = {
        'DISTANCE_SUB': DISTANCIA,
        'RESI_SUB': RESISTENCIA,
        'REAC_SUB': REACTANCIA,
        'FCONV_R': FCONV,
        'I_NOM_MED_R': I_NOM_MED,
        'I_NOM_MIN_R': I_NOM_MIN,
        'I_MAX_MED_R': I_MAX_MED,
        'I_MAX_MIN_R': I_MAX_MIN
    }
    
    return elect_ft

In [14]:
UNTRD_ELE = UNTRD_CONS.apply(get_elec_features , axis=1, result_type='expand')

In [15]:
# Agregar Variables Manualmente
UNTRD_ELE['Z_MOD_SUB'] = UNTRD_ELE.apply(lambda x: np.absolute(x.RESI_SUB - x.REAC_SUB*1j), axis=1)
UNTRD_ELE['Z_ANG_SUB'] = UNTRD_ELE.apply(lambda x: np.angle(x.RESI_SUB- x.REAC_SUB*1j, deg=True), axis=1)

## Unir variables de distancia y electricas

In [16]:
UNTRD_CONS[UNTRD_ELE.columns] = UNTRD_ELE

In [18]:
UNTRD_CONS.columns

Index(['COD_ID', 'PAC_1', 'CTMT', 'SUB', 'UNTRD_PCON', 'CTMT_PCON',
       'ROUTE_SUB', 'SID_ROUTE_SUB', 'DISTANCE_SUB', 'FCONV_R', 'I_MAX_MED_R',
       'I_MAX_MIN_R', 'I_NOM_MED_R', 'I_NOM_MIN_R', 'REAC_SUB', 'RESI_SUB',
       'Z_MOD_SUB', 'Z_ANG_SUB'],
      dtype='object')

In [19]:
# Quitar keys y listas
UNTRD_CONS.drop(['PAC_1', 'CTMT', 'SUB', 'UNTRD_PCON', 'CTMT_PCON','ROUTE_SUB', 'SID_ROUTE_SUB'], axis=1, inplace= True)

## Exportar exit point

In [20]:
filename = './data/222_ETRD_ROUTES_ELEC.pkl'
outfile = open(filename, 'wb')
pickle.dump(UNTRD_CONS,outfile)
outfile.close()